### Credit Score challenge with missing values

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.cbook import flatten
import math
from sklearn.ensemble import RandomForestClassifier
from datetime import datetime
from numpy import random

In [2]:
train_set = pd.read_csv(r'Data/train.csv')
test_set = pd.read_csv(r'Data/test.csv')

cols_Vincent = train_set.columns[:97]
cols_Duong = train_set.columns[97:]

C:\Users\boy19\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (35,43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\boy19\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (34,42) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# store all duplicated columns
duplicated_cols_dict= {}
duplicated_cols_list_full = []


# THIS FUNCTION is to drop all duplicates columns
for i, col in enumerate(train_set.columns):
    duplicated_cols_list= []
    # if the col is already one of the keys
    # then skip
    if col in duplicated_cols_list_full:
        continue
    for c in train_set.columns:
        if (train_set[col].equals(train_set[c])):
            duplicated_cols_list.append(c)
            duplicated_cols_list_full.append(c)
                   
    if (len(duplicated_cols_list) > 1):
        duplicated_cols_dict[col] = duplicated_cols_list   
    
drop_cols = set(list(flatten(list(duplicated_cols_dict.values())))) - set(duplicated_cols_dict.keys())
train_set = train_set.drop(drop_cols, axis= 1)
test_set  = test_set.drop(drop_cols, axis= 1)

In [4]:
len(train_set.columns)

149

In [5]:
train_set = train_set[train_set.columns[:74]]

In [6]:
train_set.drop(columns = ['id','Field_1', 'Field_2', 'Field_5', 'Field_6', 'Field_7', 'Field_8', 'Field_9', 'Field_11',
                         'Field_15', 'Field_18', 'Field_25', 'Field_32', 'Field_33', 'Field_34', 'Field_35', 'ngaySinh', 'diaChi', 'Field_40',
                         'Field_43', 'Field_44', 'Field_45', 'Field_46', 'Field_48', 'Field_49', 'Field_56', 'Field_61', 'Field_68', 
                          'maCv'], inplace = True)

In [7]:
train_set.head()

,label,Field_3,Field_4,Field_10,Field_12,Field_13,Field_19,Field_20,Field_21,Field_22,...,Field_67,Field_69,Field_70,Field_71,Field_72,Field_73,Field_74,Field_75,Field_76,Field_77
0,1,1.0,GH,1.0,G8,1.0,0.0,4258600.0,4.5,1.0,...,118410.0,0.00,0.49,0.245,0.49,NaN,NaN,5.0,0.0,2.0
1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,704.0,6.0,2.0,2.0
2,0,2.0,T1,1.0,NaN,1.0,0.0,5000000.0,4.5,1.0,...,1073365.0,0.58,0.77,0.675,0.19,1.0,1076.0,2.0,0.0,2.0
3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
info_Field3 = train_set.Field_3.value_counts()/train_set.Field_3.value_counts().sum()
info_Field3

2.0    0.538687
1.0    0.461313
Name: Field_3, dtype: float64

In [9]:
field3_random = random.choice(info_Field3.index, size= len(train_set), p = info_Field3.values)
train_set['Field_3'] = np.where(train_set['Field_3'].isnull(),\
            field3_random,\
            train_set['Field_3'])

In [10]:
def Fill_Cat_Var(df, col_name, seed_val = 76):
    random.seed(seed_val)
    for name in col_name:
        info = df[name].value_counts()/df[name].value_counts().sum()
        random_val = random.choice(info.index, size= len(df), p = info.values)
        df[name] = np.where(df[name].isnull(), random_val, df[name])

In [11]:
Fill_Cat_Var(train_set, ['Field_3', 'Field_4', 'Field_10', 'Field_12', 'Field_13', 'Field_19', 'Field_20', 'Field_21', 'Field_22',
                        'Field_23', 'Field_27', 'Field_28', 'Field_29', 'namSinh', 'gioiTinh', 'Field_36', 'Field_38', 
                         'Field_39', 'Field_41', 'Field_42', 'Field_47', 'Field_50', 'Field_51', 'Field_53', 'Field_54',
                        'Field_55', 'Field_58', 'Field_59', 'Field_60', 'Field_62', 'Field_63','Field_64', 'Field_65','Field_66', 
                        'Field_67', 'Field_69', 'Field_70', 'Field_71', 'Field_72', 'Field_73', 'Field_74', 'Field_75', 
                         'Field_76', 'Field_77'])


In [12]:
train_set.columns

Index(['label', 'Field_3', 'Field_4', 'Field_10', 'Field_12', 'Field_13',
       'Field_19', 'Field_20', 'Field_21', 'Field_22', 'Field_23', 'Field_27',
       'Field_28', 'Field_29', 'namSinh', 'gioiTinh', 'Field_36', 'Field_38',
       'Field_39', 'Field_41', 'Field_42', 'Field_47', 'Field_50', 'Field_51',
       'Field_53', 'Field_54', 'Field_55', 'Field_58', 'Field_59', 'Field_60',
       'Field_62', 'Field_63', 'Field_64', 'Field_65', 'Field_66', 'Field_67',
       'Field_69', 'Field_70', 'Field_71', 'Field_72', 'Field_73', 'Field_74',
       'Field_75', 'Field_76', 'Field_77'],
      dtype='object')

In [13]:
train_set.isnull().sum().sum()

0

In [14]:
train_set.to_csv('Duong_Dumbass.csv', index = False)